In [39]:
import serial
import time

SERIAL_PORT = "/dev/ttyUSB1"
# after writing commands a pause is needed
CMD_WRITE_PAUSE = 0.2

MM12_READ_INFOS   = b'UU\x55\x55\x00\x00\x00\xaa'
MM12_READ_DISPLAY = b'\x55\x55\x01\x00\x00\xab'

try:
    serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=2)

except serial.SerialException:
    print("Could not open serial port '{0}'!".format(SERIAL_PORT))
    raise

In [17]:
#message_bytes = bytes.fromhex(MM12_READ_INFOS)

MM12_READ_INFOS   = b'\x55\x55\x00\x00\x00\xaa'
print(MM12_READ_INFOS)

#serial.write(message_bytes)
#time.sleep(CMD_WRITE_PAUSE)

b'UU\x00\x00\x00\xaa'


In [25]:
# read setting success
#print(serial.read_all().decode('utf-8'))
print(serial.read_all())

b''


In [10]:
message_bytes = bytes.fromhex("0111050200013F0804")
print(message_bytes.decode('hex'))

LookupError: 'hex' is not a text encoding; use codecs.decode() to handle arbitrary codecs

In [33]:
from binascii import unhexlify

string_a = '0281E1B1'
string_b = '5555000000aa'
string_c = '0000aa'

print(unhexlify(string_b))

b'UU\x00\x00\x00\xaa'


In [40]:
my_bytestring3 = bytes([0x55, 0x55, 0x00, 0x00, 0x00, 0xaa])

print(my_bytestring3)

b'UU\x00\x00\x00\xaa'


In [41]:
serial.write(my_bytestring3)

6

In [59]:
print(serial.read_all())

b''


In [61]:
MM12_READ_INFOS_1   = b'\x55\x55\x01'
MM12_READ_INFOS_2   = b'\x00\x00\xab'

serial.write(MM12_READ_INFOS_1)
time.sleep(CMD_WRITE_PAUSE)
serial.write(MM12_READ_INFOS_2)
time.sleep(CMD_WRITE_PAUSE)

while 1:
    print(serial.read_all())
    time.sleep(CMD_WRITE_PAUSE)

b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''
b''


KeyboardInterrupt: 